# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

# TENSORFLOW
from tensorflow import keras
from tensorflow.keras import layers

2023-02-16 05:04:55.845940: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Data

In [2]:
data_path='../MATERIALS/DATA/processed_data/transaction_data.csv'
df       =pd.read_csv(data_path).drop('Unnamed: 0', axis=1)

In [3]:
data=pd.to_datetime(df.DATA)
np.unique(data.dt.year) # ONLY ONE YEAR!

array([2022])

In [4]:
df['DATA']=pd.to_datetime(df.DATA)

# Further Time Series Data Processing

Idea is to have a DataFrame for each client where every row is the sum of all variables within a week. 

We must add ghost rows on each 1st day of the week, to make sure all DataFrames have the same size.

In [5]:
ghost_index=pd.date_range(start='2022-01-01', end='2022-05-1', freq='W')

Start be creating a DataFrame for each client

In [6]:
client_dataframes=dict()
client_target    =dict()
unique_clients   = np.unique(df.ID)

for client_ID in unique_clients:
    client_dataframes[client_ID]=df[df.ID==client_ID].set_index('DATA').drop(['ID', 'target'], axis=1)
    client_target[client_ID]=np.mean(df[df.ID==client_ID].target).astype(int)

In [7]:
#TEST TO GET START AND END DAY OF COUNTING
#client=client_dataframes[171]
#client.sort_values(by='DATA').index[-1]

Finally, resample on a weekly basis

In [8]:
# TAKES TOO LONG TO RUN - POORLY OPTIMIZED CODE
for client_ID in unique_clients:
    client=client_dataframes[client_ID]
    for index in ghost_index:
        client.loc[len(client)] = 0                             # Adds ghost row on index=len(client)
        client.rename(index={len(client)-1:index},inplace=True) # Changes ghost row index to time data
    client_dataframes[client_ID]=client.resample('W').sum()

Check all client DataFrames have same dimentions

In [9]:
check=client_dataframes[1].shape

sum([client_dataframes[client_ID].shape!=check for client_ID in unique_clients])

0

save DataFrame as .csv

In [10]:
dataset_time=pd.DataFrame({})

for client_ID in unique_clients:
    
    client=client_dataframes[client_ID]
    client[['ID', 'target']]=[client_ID, client_target[client_ID]]
    
    frames = [dataset_time, client]
    dataset_time = pd.concat(frames)

In [11]:
dataset_time.reset_index(inplace=True)
dataset_time.index=dataset_time.ID
dataset_time.drop('ID', inplace=True, axis=1)

In [12]:
dataset_time.to_csv('../MATERIALS/DATA/processed_data/processed_time_dataset.csv')

# LSTM Build Model

#  Transformer Build Model